In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bipolar_disorder"
cohort = "GSE53987"

# Input paths
in_trait_dir = "../../input/GEO/Bipolar_disorder"
in_cohort_dir = "../../input/GEO/Bipolar_disorder/GSE53987"

# Output paths
out_data_file = "../../output/preprocess/Bipolar_disorder/GSE53987.csv"
out_gene_data_file = "../../output/preprocess/Bipolar_disorder/gene_data/GSE53987.csv"
out_clinical_data_file = "../../output/preprocess/Bipolar_disorder/clinical_data/GSE53987.csv"
json_path = "../../output/preprocess/Bipolar_disorder/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll implement a complete solution for extracting clinical features from the sample characteristics dictionary provided in the previous step.

```python
# 1. Gene Expression Data Availability
# Check if the series contains gene expression data (vs miRNA/methylation)
# The background information describes this as "Microarray profiling" with "RNA isolated and hybridized" 
# and U133_Plus2 Affymetrix chips, which indicates gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability for trait, age, and gender

# Trait (Bipolar disorder) - from key 7: 'disease state'
trait_row = 7  # Key for 'disease state' which includes bipolar disorder

# Age - from key 0
age_row = 0

# Gender - from key 1
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert trait value for bipolar disorder to binary:
    1 for bipolar disorder, 0 for control/other disorders
    """
    if value is None or ':' not in value:
        return None
    
    # Extract value after the colon
    value = value.split(':', 1)[1].strip().lower()
    
    # 1 for bipolar disorder, 0 for others
    if value == 'bipolar disorder':
        return 1
    elif value in ['control', 'major depressive disorder', 'schizophrenia']:
        return 0
    else:
        return None

def convert_age(value):
    """
    Convert age value to continuous (integer)
    """
    if value is None or ':' not in value:
        return None
    
    # Extract value after the colon
    value = value.split(':', 1)[1].strip()
    
    # Try to convert to integer
    try:
        return int(value)
    except ValueError:
        return None

def convert_gender(value):
    """
    Convert gender value to binary:
    0 for female (F), 1 for male (M)
    """
    if value is None or ':' not in value:
        return None
    
    # Extract value after the colon
    value = value.split(':', 1)[1].strip().upper()
    
    if value == 'F':
        return 0
    elif value == 'M':
        return 1
    else:
        return None

# 3. Save Metadata - initial validation
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info (initial validation)
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait_row is not None, extract and process clinical features
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the task
    sample_chars = {
        0: ['age: 52', 'age: 50', 'age: 28', 'age: 55', 'age: 58', 'age: 49', 'age: 42', 'age: 43', 'age: 40', 'age: 39', 
            'age: 45', 'age: 65', 'age: 51', 'age: 48', 'age: 36', 'age: 22', 'age: 41', 'age: 68', 'age: 53', 'age: 26', 
            'age: 62', 'age: 29', 'age: 54', 'age: 44', 'age: 47', 'age: 59', 'age: 34', 'age: 25', 'age: 46', 'age: 37'],
        1: ['gender: M', 'gender: F'],
        2: ['race: W', 'race: B'],
        3: ['pmi: 23.5', 'pmi: 11.7', 'pmi: 22.3', 'pmi: 17.5', 'pmi: 27.7', 'pmi: 27.4', 'pmi: 21.5', 'pmi: 31.2', 
            'pmi: 31.9', 'pmi: 12.1', 'pmi: 18.5', 'pmi: 22.2', 'pmi: 27.2', 'pmi: 12.5', 'pmi: 8.9', 'pmi: 24.2', 
            'pmi: 18.1', 'pmi: 7.8', 'pmi: 14.5', 'pmi: 28', 'pmi: 20.1', 'pmi: 22.6', 'pmi: 22.7', 'pmi: 16.6', 
            'pmi: 15.4', 'pmi: 21.2', 'pmi: 21.68', 'pmi: 24.5', 'pmi: 13.8', 'pmi: 11.8'],
        4: ['ph: 6.7', 'ph: 6.4', 'ph: 6.3', 'ph: 6.8', 'ph: 6.2', 'ph: 6.5', 'ph: 7.1', 'ph: 6.6', 'ph: 6.9', 'ph: 6.1', 
            'ph: 7.3', 'ph: 5.97', 'ph: 6.35', 'ph: 6.73', 'ph: 7.14', 'ph: 6.63', 'ph: 6.61', 'ph: 6.23', 'ph: 6.19', 
            'ph: 6.27', 'ph: 6.58', 'ph: 6.07', 'ph: 6.22', 'ph: 6.56', 'ph: 6.68', 'ph: 6.18', 'ph: 6.25'],
        5: ['rin: 6.3', 'rin: 6.8', 'rin: 7.7', 'rin: 7.6', 'rin: 7', 'rin: 8.2', 'rin: 5.6', 'rin: 7.4', 'rin: 6.5', 
            'rin: 7.9', 'rin: 8.1', 'rin: 6.6', 'rin: 7.8', 'rin: 7.2', 'rin: 8', 'rin: 7.1', 'rin: 8.5', 'rin: 7.3', 
            'rin: 6.1', 'rin: 7.5', 'rin: 6.2', 'rin: 5.5', 'rin: 8.4', 'rin: 6', 'rin: 6.9', 'rin: 6.7', 'rin: 6.4', 
            'rin: 8.6', 'rin: 8.3', 'rin: 8.7'],
        6: ['tissue: hippocampus', 'tissue: Pre-frontal cortex (BA46)', 'tissue: Associative striatum'],
        7: ['disease state: bipolar disorder', 'disease state: control', 'disease state: major depressive disorder', 
            'disease state: schizophrenia']
    }
    
    # Convert dictionary to DataFrame
    clinical_data = pd.DataFrame(sample_chars)
    
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import numpy as np
import json
import gzip
import re
from typing import Callable, Optional, Dict, Any

# First, let's analyze the dataset by reading the compressed GEO series matrix file
raw_data_path = os.path.join(in_cohort_dir, "GSE53987_series_matrix.txt.gz")

# Read the compressed file and extract sample characteristics
sample_characteristics = {}
platform_info = ""
is_gene_available = False

try:
    with gzip.open(raw_data_path, 'rt') as file:
        for line in file:
            if line.startswith('!Sample_characteristics_ch1'):
                parts = line.strip().split('\t')
                for i, part in enumerate(parts[1:], 1):
                    if ':' in part:
                        key, value = part.split(':', 1)
                        key = key.strip()
                        if key not in sample_characteristics:
                            sample_characteristics[key] = []
                        sample_characteristics[key].append(value.strip())
            # Check for platform to determine if gene expression data is available
            elif line.startswith('!Platform_technology'):
                platform_info = line.strip()
            # If we see gene expression related lines, mark as available
            elif line.startswith('!platform_table_begin') or 'gene' in line.lower() or 'expression' in line.lower():
                is_gene_available = True
            # Break after reading a significant portion to improve efficiency
            elif line.startswith('!series_matrix_table_begin'):
                # We've reached the data matrix, stop reading
                break
                
    print("Sample characteristics found:")
    for key, values in sample_characteristics.items():
        unique_values = set(values)
        print(f"{key}: {unique_values}")
    print(f"Platform info: {platform_info}")
    
except Exception as e:
    print(f"Error reading series matrix file: {e}")
    sample_characteristics = {}
    is_gene_available = False

# Parse clinical data from sample characteristics
clinical_data = None
if sample_characteristics:
    # Convert sample characteristics to dataframe for geo_select_clinical_features function
    clinical_rows = []
    for key, values in sample_characteristics.items():
        row = [key] + values
        clinical_rows.append(row)
    
    # Create dataframe with header being sample IDs
    sample_ids = [f"Sample_{i+1}" for i in range(len(list(sample_characteristics.values())[0]))]
    clinical_data = pd.DataFrame(clinical_rows, columns=['Feature'] + sample_ids)
    print("\nClinical data preview:")
    print(clinical_data.head())

# Determine trait, age, and gender rows based on the sample characteristics
trait_row = None
age_row = None
gender_row = None

# Find trait row
disease_keywords = ['diagnosis', 'disease', 'disorder', 'condition', 'group', 'subject', 'bipolar']
for i, feature in enumerate(clinical_data['Feature'] if clinical_data is not None else []):
    feature_lower = feature.lower()
    if any(keyword in feature_lower for keyword in disease_keywords):
        # Check if there's more than one unique value (excluding None, nan, etc.)
        unique_values = set(v for v in clinical_data.iloc[i, 1:] if v and not pd.isna(v))
        if len(unique_values) > 1:
            trait_row = i
            print(f"Found trait row: {i} - {feature}")
            print(f"Unique values: {unique_values}")
            break

# Find age row
age_keywords = ['age', 'years']
for i, feature in enumerate(clinical_data['Feature'] if clinical_data is not None else []):
    feature_lower = feature.lower()
    if any(keyword in feature_lower for keyword in age_keywords):
        # Check if there's variation in age values
        unique_values = set(v for v in clinical_data.iloc[i, 1:] if v and not pd.isna(v))
        if len(unique_values) > 1:
            age_row = i
            print(f"Found age row: {i} - {feature}")
            print(f"Sample unique values: {list(unique_values)[:5]}")
            break

# Find gender row
gender_keywords = ['gender', 'sex']
for i, feature in enumerate(clinical_data['Feature'] if clinical_data is not None else []):
    feature_lower = feature.lower()
    if any(keyword in feature_lower for keyword in gender_keywords):
        # Check if there's variation in gender values
        unique_values = set(v for v in clinical_data.iloc[i, 1:] if v and not pd.isna(v))
        if len(unique_values) > 1:
            gender_row = i
            print(f"Found gender row: {i} - {feature}")
            print(f"Unique values: {unique_values}")
            break

# Define conversion functions based on identified rows
def convert_trait(value):
    if value is None or pd.isna(value):
        return None
    
    value_lower = value.lower()
    if 'bipolar' in value_lower or 'bd' in value_lower or 'bpd' in value_lower:
        return 1  # Has bipolar disorder
    elif 'control' in value_lower or 'normal' in value_lower or 'healthy' in value_lower or 'con' in value_lower:
        return 0  # Control
    else:
        # If not clear, return None
        return None

def convert_age(value):
    if value is None or pd.isna(value):
        return None
    
    # Try to extract numbers
    numbers = re.findall(r'\d+\.?\d*', str(value))
    if numbers:
        try:
            return float(numbers[0])
        except:
            return None
    return None

def convert_gender(value):
    if value is None or pd.isna(value):
        return None
    
    value_lower = value.lower()
    if 'female' in value_lower or 'f' == value_lower:
        return 0
    elif 'male' in value_lower or 'm' == value_lower:
        return 1
    else:
        return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait_row is not None and clinical_data exists
if is_trait_available and clinical_data is not None:
    # Use the provided function to select clinical features
    clinical_features = geo_select_clinical_features(
        clinical_data, 
        trait=trait, 
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted features
    print("\nPreview of extracted clinical features:")
    print(preview_df(clinical_features))
    
    # Save the clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 5: Gene Identifier Review

In [ ]:
# The gene/probe identifiers (e.g., '1007_s_at', '1053_at') appear to be Affymetrix probe IDs 
# rather than standard human gene symbols (which would be like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check if there are any columns that might contain gene information
sample_row = gene_annotation.iloc[0].to_dict()
print("\nFirst row as dictionary:")
for col, value in sample_row.items():
    print(f"{col}: {value}")

# Check if IDs in gene_data match IDs in annotation
print("\nComparing gene data IDs with annotation IDs:")
print("First 5 gene data IDs:", gene_data.index[:5].tolist())
print("First 5 annotation IDs:", gene_annotation['ID'].head().tolist())

# Properly check for exact ID matches between gene data and annotation
gene_data_ids = set(gene_data.index)
annotation_ids = set(gene_annotation['ID'].astype(str))
matching_ids = gene_data_ids.intersection(annotation_ids)
id_match_percentage = len(matching_ids) / len(gene_data_ids) * 100 if len(gene_data_ids) > 0 else 0

print(f"\nExact ID match between gene data and annotation:")
print(f"Matching IDs: {len(matching_ids)} out of {len(gene_data_ids)} ({id_match_percentage:.2f}%)")

# Check which columns might contain gene symbols for mapping
potential_gene_symbol_cols = [col for col in gene_annotation.columns 
                             if any(term in col.upper() for term in ['GENE', 'SYMBOL', 'NAME'])]
print(f"\nPotential columns for gene symbols: {potential_gene_symbol_cols}")

# Check if the identified columns contain non-null values
for col in potential_gene_symbol_cols:
    non_null_count = gene_annotation[col].notnull().sum()
    non_null_percent = non_null_count / len(gene_annotation) * 100
    print(f"Column '{col}': {non_null_count} non-null values ({non_null_percent:.2f}%)")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in gene_annotation hold the probe IDs and gene symbols
# From previous analysis, 'ID' contains the Affymetrix probe IDs and 'Gene Symbol' contains the gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

print(f"Using {prob_col} as probe identifier column and {gene_col} as gene symbol column")

# 2. Get gene mapping dataframe by extracting these two columns
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Mapping data shape: {mapping_data.shape}")
print(f"Sample of mapping data (first 5 rows):")
print(mapping_data.head())

# Check how many probes map to multiple genes
mapping_data['Gene'] = mapping_data['Gene'].astype(str)
multi_gene_probes = mapping_data[mapping_data['Gene'].str.contains('///')]
print(f"\nNumber of probes mapping to multiple genes: {len(multi_gene_probes)}")
print(f"Sample of probes with multiple genes (first 5):")
if len(multi_gene_probes) > 0:
    print(multi_gene_probes.head())

# 3. Apply gene mapping to convert probe-level data to gene expression data
# The library function apply_gene_mapping handles the many-to-many mapping
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print(f"First 10 gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. We already normalized gene symbols in the gene expression data in step 6
# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# First, let's extract the clinical features properly
# Based on the sample characteristics dictionary from step 1:
# {0: ['tissue: Blood'], 1: ['illness: Major Depressive Disorder'], 2: ['age: 16', 'age: 13', 'age: 12', 'age: 14', 'age: 17', 'age: 15'], 3: ['Sex: female', 'Sex: male']}

def convert_trait(value):
    """Convert depression status to binary format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "major depressive disorder" in value:
        return 1  # Has depression
    return 0  # Control/no depression

def convert_age(value):
    """Convert age data to continuous format."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip()
    try:
        return float(value)
    except:
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    if not isinstance(value, str):
        return None
    value = value.split(": ")[-1].strip().lower()
    if "female" in value:
        return 0
    elif "male" in value:
        return 1
    return None

# Get clinical data using the correct row index identified in step 1
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Using row 1 for depression status (major depressive disorder)
    convert_trait=convert_trait,
    age_row=2,    # Age data is in row 2
    convert_age=convert_age,
    gender_row=3, # Gender data is in row 3
    convert_gender=convert_gender
)

print(f"Selected clinical data shape: {selected_clinical_df.shape}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save clinical data for future reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
print(linked_data.iloc[:5, :5] if not linked_data.empty else "Linked data is empty")

# 3. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# 4. Check for bias in features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from blood samples of children and adolescents with Major Depressive Disorder, before and after Fluoxetine treatment."
)

# 6. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for analysis. No linked data file saved.")